In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re

In [ ]:
https://indeks.kompas.com/news/2018-07-19/2

In [223]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    if cat_link == 'otomotif':
        url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    else :
        url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.findAll('div', class_="latest--indeks mt2 clearfix")
    for post in contentDiv:
        link = [post.find('a', href=True)['href'], category]
        links.append(link)
        
    el_page = soup.find('div', class_="paging__wrap clearfix")
    if el_page:
        a_page = soup.find('div', class_="paging__wrap clearfix").findAll('div', class_='paging__item')[-1].find('a')
        if 'data-ci-pagination-page' not in a_page:
            max_page = page
        else:
            max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

In [224]:
def getDetailBerita(links):
    articles = {}
    for link in links:
        #link
        articles['url'] = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")
        
        #articleid
        articles['id'] = int(soup.find("meta", attrs={'name':'cXenseParse:articleid'})['content'])
        
        #category
        articles['category'] = link[1]
        
        #extract subcategory from breadcrumb
        bc = soup.find('ul', class_="breadcrumb__wrap")
        articles['subcategory'] = bc.findAll('li')[2].text
        
        #article
        article = soup.find("div", class_="read__content")
        
        #extract date
        pubdate = soup.find("meta", attrs={"name":"content_PublishedDate"})['content']
        pubdate = pubdate.strip(' \t\n\r')
        articles['pubdate']=datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")
        
        #extract author
        articles['author'] = soup.find('div', class_="read__author").text
        
        #extract title
        articles['title'] = soup.find('h1', class_="read__title").text      

        #source
        articles['source'] = 'detik'
        
        #extract comments count
        #articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
        
        #extract tags
        tags = soup.find("meta", attrs={'name':'content_tag'})['content']
        articles['tags'] = ','.join([x.text for x in tags])
        
        #extract images
        articles['images'] = soup.find('meta', attrs={'property':'og:image'})['content']
        
       
        #hapus link sisip
#         for link in detail.findAll('table', class_="linksisip"):
#             link.decompose()
        
        #hapus video sisip
#         for tag in detail.findAll('div', class_="detail_tag"):
#             tag.decompose()
        
        #extract content
        detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = content
        
        return articles

In [225]:
links = []

In [226]:
links

[]

In [227]:
detail = getIndeksLink(links, 1, 'otomotif', 'otomotif')

page  1
https://indeks.kompas.com/otomotif/2018-07-27/1


In [147]:
# links = getIndeksLink(links, 1, news, news)

In [195]:
response = requests.get("https://indeks.kompas.com/otomotif/2018-07-27/1")
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")

In [200]:
max_page = soup.find('div', class_="paging__wrap clearfix").findAll('div', class_='paging__item')[-1].find('a')['data-ci-pagination-page'].replace('\n', '').strip(' ')


In [228]:
detail

[['http://otomotif.kompas.com/read/2018/07/27/162926715/polisi-tindak-pengendara-dengan-modus-akali-ganjil-genap',
  'otomotif']]